In [1]:
from nltk.corpus import wordnet as wn

In [2]:
def pre_processing(sentence):
    return remove_stopwords(remove_punctuation(tokenize(sentence)))

In [3]:
def remove_stopwords(words_list):
    stopwords_list = get_stopwords()
    new_words_list = []
    for word in words_list:
        word_lower = word.lower()
        if word_lower not in stopwords_list:
            new_words_list.append(word_lower)
    return new_words_list

In [4]:
import re

def remove_punctuation(sentence):
    return re.sub(r'[^\w\s]', '', sentence)


In [5]:
def all_hypernym_paths(word):
    
    def_lens = []
    
    for syn in wn.synsets(word):

        single_path = []

        hyp_path = syn.hypernym_paths()

        for i in range (0, len(hyp_path[0])):
            single_path.append(len((hyp_path[0][i].definition()).split()))

        def_lens.append(single_path)
        
    return def_lens

In [6]:
all_hypernym_paths("courage")

[[17, 11, 9, 7, 18, 9, 15]]

In [7]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
#dato in input un synset, la funzione ritorna la lista di iperonimi e iponimi
